<a href="https://colab.research.google.com/github/MohammedYaseen97/idiomaticity-detection/blob/main/adapter_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction

This notebook provides a baseline for each setting in [Subtask A of SemEval 2022 Task 2](https://sites.google.com/view/semeval2022task2-idiomaticity#h.qq7eefmehqf9). In addition this provides some helpful pre-processing scripts that you are free to use with your experiments. 

Please start by stepping through this notebook so you have a clear idea as to what is expected of the task and what you need to submit. 

These baselines are based on the results described in the paper “[AStitchInLanguageModels: Dataset and Methods for the Exploration of Idiomaticity in Pre-Trained Language Models](https://arxiv.org/abs/2109.04413)”. 

## Zero-shot setting: Methodology 

Note that in the zero-shot setting you are NOT allowed to train the model using the one-shot data. 

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). This is based on the results presented in the dataset paper. 

We use Multilingual BERT for this setting.

## One-shot setting: Methodology

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. Again, this is based on the results presented in the dataset paper. 

We also use Multilingual BERT for this setting.


In [1]:
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cuda


# Setup 

Download the Task data and evaluation scripts

In [2]:
!git clone https://github.com/H-TayyarMadabushi/SemEval_2022_Task2-idiomaticity.git

Cloning into 'SemEval_2022_Task2-idiomaticity'...
remote: Enumerating objects: 123, done.
remote: Counting objects: 100% (123/123), done.
remote: Compressing objects: 100% (106/106), done.
remote: Total 123 (delta 48), reused 61 (delta 15), pack-reused 0
Receiving objects: 100% (123/123), 2.50 MiB | 10.77 MiB/s, done.
Resolving deltas: 100% (48/48), done.


Download the “AStitchInLanguageModels” code which we make use of. 

In [3]:
!git clone https://github.com/H-TayyarMadabushi/AStitchInLanguageModels.git

Cloning into 'AStitchInLanguageModels'...
remote: Enumerating objects: 1030, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 1030 (delta 11), reused 4 (delta 4), pack-reused 1013
Receiving objects: 100% (1030/1030), 79.59 MiB | 45.20 MiB/s, done.
Resolving deltas: 100% (394/394), done.


Download and install an editable version of huggingfaces transformers. 

In [4]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers/
!pip install --editable .
%cd /content/ 

Cloning into 'transformers'...
remote: Enumerating objects: 106376, done.
remote: Counting objects: 100% (228/228), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 106376 (delta 96), reused 144 (delta 46), pack-reused 106148
Receiving objects: 100% (106376/106376), 98.62 MiB | 29.42 MiB/s, done.
Resolving deltas: 100% (78523/78523), done.
/content/transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/transformers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 120 kB 32.3 MB/s 
     |████████████████████████████████| 6.6 MB 58.4 MB/s 
  Running setup.py develop for transformers
/content


Required for run_glue ... 

In [5]:
## run_glue needs this. 
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 28.3 MB/s 
     |████████████████████████████████| 115 kB 63.4 MB/s 
     |████████████████████████████████| 212 kB 44.7 MB/s 
     |████████████████████████████████| 127 kB 76.5 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


Editable install requires runtime restart unless we do this. 

In [6]:
import site
site.main()


# Imports and Helper functions

In [7]:
import os
import csv

from pathlib import Path

In [8]:
def load_csv( path, delimiter=',' ) : 
  header = None
  data   = list()
  with open( path, encoding='utf-8') as csvfile:
    reader = csv.reader( csvfile, delimiter=delimiter ) 
    for row in reader : 
      if header is None : 
        header = row
        continue
      data.append( row ) 
  return header, data


In [9]:
def write_csv( data, location ) : 
  with open( location, 'w', encoding='utf-8') as csvfile:
    writer = csv.writer( csvfile ) 
    writer.writerows( data ) 
  print( "Wrote {}".format( location ) ) 
  return


# Pre-process: Create train and dev and evaluation data in required format

In the zero-shot setting, we choose to include the context (the sentences preceding and succeeding the one containing the idioms). We do not add the idiom as an additional feature (in the “second input sentence”). 

In the one shot setting, we train the model on both the zero-shot and one-shot data. In this setting, we exclude the context (the sentences preceding and succeeding the one containing the idioms) and also add the idiom as an additional feature in the “second sentence”. 


## Functions for pre-processing

### _get_train_data

This function generates training data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 


In [10]:
def _get_train_data( data_location, file_name, include_context, include_idiom ) :
    
    file_name = os.path.join( data_location, file_name ) 

    header, data = load_csv( file_name )

    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]
        
    # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
    out_data = list()
    for elem in data :
        label     = elem[ header.index( 'Label'  ) ]
        sentence1 = elem[ header.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ header.index( 'Previous' ) ], elem[ header.index( 'Target' ) ], elem[ header.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ header.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        out_data.append( this_row )
        assert len( out_header ) == len( this_row )
    return [ out_header ] + out_data

### _get_dev_eval_data

This function generates training dev and eval data in the format required by the huggingface’s example script. It will include and exclude the MWE and the context based on parameters. 

Additionally, if there is no gold label provides (as in the case of eval) it will generate a file that can be used to generate predictions.


In [11]:
def _get_dev_eval_data( data_location, input_file_name, gold_file_name, include_context, include_idiom ) :

    input_headers, input_data = load_csv( os.path.join( data_location, input_file_name ) )
    gold_header  = gold_data = None
    if not gold_file_name is None : 
        gold_header  , gold_data  = load_csv( os.path.join( data_location, gold_file_name  ) )
        assert len( input_data ) == len( gold_data )

    # ['ID', 'Language', 'MWE', 'Previous', 'Target', 'Next']
    # ['ID', 'DataID', 'Language', 'Label']
    
    out_header = [ 'label', 'sentence1' ]
    if include_idiom :
        out_header = [ 'label', 'sentence1', 'sentence2' ]

    out_data = list()
    for index in range( len( input_data ) ) :
        label = 1
        if not gold_file_name is None : 
            this_input_id = input_data[ index ][ input_headers.index( 'ID' ) ]
            this_gold_id  = gold_data [ index ][ gold_header  .index( 'ID' ) ]
            assert this_input_id == this_gold_id
            
            label     = gold_data[ index ][ gold_header.index( 'Label'  ) ]
            
        elem      = input_data[ index ]
        sentence1 = elem[ input_headers.index( 'Target' ) ]
        if include_context :
            sentence1 = ' '.join( [ elem[ input_headers.index( 'Previous' ) ], elem[ input_headers.index( 'Target' ) ], elem[ input_headers.index( 'Next' ) ] ] )
        this_row = None
        if not include_idiom :
            this_row = [ label, sentence1 ] 
        else :
            sentence2 = elem[ input_headers.index( 'MWE' ) ]
            this_row = [ label, sentence1, sentence2 ]
        assert len( out_header ) == len( this_row ) 
        out_data.append( this_row )
        

    return [ out_header ] + out_data


### create_data

This function generates the training, development and evaluation data. 


In [12]:
"""
Based on the results presented in `AStitchInLanguageModels' we work with not including the idiom for the zero shot setting and including it in the one shot setting.
"""
def create_data( input_location, output_location, lang=None) :

    
    ## Zero shot data
    train_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv' if lang is None else 'train_zero_shot_'+lang.lower()+'.csv',
        include_context = True,
        include_idiom   = False
    )
    if lang:
      write_csv( train_data, os.path.join( output_location, 'ZeroShot', lang, 'train.csv' ) )
    else:  
      write_csv( train_data, os.path.join( output_location, 'ZeroShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv' if lang is None else 'dev_'+lang.lower()+'.csv',
        gold_file_name   = 'dev_gold.csv' if lang is None else 'dev_gold_'+lang.lower()+'.csv', 
        include_context  = True,
        include_idiom    = False
    ) 
    if lang:
      write_csv( dev_data, os.path.join( output_location, 'ZeroShot', lang, 'dev.csv' ) )
    else:  
      write_csv( dev_data, os.path.join( output_location, 'ZeroShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv' if lang is None else 'eval_'+lang.lower()+'.csv',
        gold_file_name   = None , ## Don't have gold evaluation file -- submit to CodaLab
        include_context  = True,
        include_idiom    = False
    )
    if lang:
      write_csv( eval_data, os.path.join( output_location, 'ZeroShot', lang, 'eval.csv' ) )
    else:  
      write_csv( eval_data, os.path.join( output_location, 'ZeroShot', 'eval.csv' ) )


    ## OneShot Data (combine both for training)
    train_zero_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_zero_shot.csv' if lang is None else 'train_zero_shot_'+lang.lower()+'.csv',
        include_context = False,
        include_idiom   = True
    )
    train_one_data = _get_train_data(
        data_location   = input_location,
        file_name       = 'train_one_shot.csv' if lang is None else 'train_one_shot_'+lang.lower()+'.csv',
        include_context = False,
        include_idiom   = True
    )

    assert train_zero_data[0] == train_one_data[0] ## Headers
    train_data = train_one_data + train_zero_data[1:]
    if lang:
      write_csv( train_data, os.path.join( output_location, 'OneShot', lang, 'train.csv' ) )
    else:  
      write_csv( train_data, os.path.join( output_location, 'OneShot', 'train.csv' ) )
    
    dev_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'dev.csv' if lang is None else 'dev_'+lang.lower()+'.csv',
        gold_file_name   = 'dev_gold.csv' if lang is None else 'dev_gold_'+lang.lower()+'.csv', 
        include_context  = False,
        include_idiom    = True
    )        
    if lang:
      write_csv( dev_data, os.path.join( output_location, 'OneShot', lang, 'dev.csv' ) )
    else:  
      write_csv( dev_data, os.path.join( output_location, 'OneShot', 'dev.csv' ) )
    
    eval_data = _get_dev_eval_data(
        data_location    = input_location,
        input_file_name  = 'eval.csv' if lang is None else 'eval_'+lang.lower()+'.csv',
        gold_file_name   = None,
        include_context  = False,
        include_idiom    = True
    )
    if lang:
      write_csv( eval_data, os.path.join( output_location, 'OneShot', lang, 'eval.csv' ) )
    else:  
      write_csv( eval_data, os.path.join( output_location, 'OneShot', 'eval.csv' ) )

    return

## Setup and Create data

In [13]:
!ls 

AStitchInLanguageModels  SemEval_2022_Task2-idiomaticity
sample_data		 transformers


In [14]:
def _get_lang( data_location, file_name, lang ) :
  # Filter csv files by language
    
  file_name = os.path.join( data_location, file_name ) 

  header, data = load_csv( file_name )

  # ['DataID', 'Language', 'MWE', 'Setting', 'Previous', 'Target', 'Next', 'Label']
  out_data = list()
  for elem in data :
      if elem[ header.index( 'Language'  ) ] == lang:
        out_data.append( elem )
  # write_csv( eval_data, os.path.join( data_location, file_name+"_"+lang.tolower()+".csv" ) )
  return [header] + out_data

In [15]:
def create_lang_input(file_location, lang):
  train0_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'train_zero_shot.csv',
      lang=lang
  )
  write_csv( train0_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "train_zero_shot_"+lang.lower()+".csv" ) )

  train1_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'train_one_shot.csv',
      lang=lang
  )
  write_csv( train1_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "train_one_shot_"+lang.lower()+".csv" ) )

  dev_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'dev.csv',
      lang=lang
  )
  write_csv( dev_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "dev_"+lang.lower()+".csv" ) )

  dev_gold_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'dev_gold.csv',
      lang=lang
  )
  write_csv( dev_gold_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "dev_gold_"+lang.lower()+".csv" ) )

  eval_lang_data = _get_lang(
      data_location    = 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/',
      file_name  = 'eval.csv',
      lang=lang
  )
  write_csv( eval_lang_data, os.path.join( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', "eval_"+lang.lower()+".csv" ) )

In [27]:
language = 'EN'

In [28]:
create_lang_input('SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', lang=language)

Wrote SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_zero_shot_en.csv
Wrote SemEval_2022_Task2-idiomaticity/SubTaskA/Data/train_one_shot_en.csv
Wrote SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_en.csv
Wrote SemEval_2022_Task2-idiomaticity/SubTaskA/Data/dev_gold_en.csv
Wrote SemEval_2022_Task2-idiomaticity/SubTaskA/Data/eval_en.csv


In [29]:
outpath = 'Data'

if language:    
  Path( os.path.join( outpath, 'ZeroShot', language ) ).mkdir(parents=True, exist_ok=True)
  Path( os.path.join( outpath, 'OneShot', language ) ).mkdir(parents=True, exist_ok=True)
else:    
  Path( os.path.join( outpath, 'ZeroShot') ).mkdir(parents=True, exist_ok=True)
  Path( os.path.join( outpath, 'OneShot') ).mkdir(parents=True, exist_ok=True)

create_data( 'SemEval_2022_Task2-idiomaticity/SubTaskA/Data/', outpath, lang=language)

Wrote Data/ZeroShot/EN/train.csv
Wrote Data/ZeroShot/EN/dev.csv
Wrote Data/ZeroShot/EN/eval.csv
Wrote Data/OneShot/EN/train.csv
Wrote Data/OneShot/EN/dev.csv
Wrote Data/OneShot/EN/eval.csv


# Zero Shot Setting

## Baseline EN : Monolingual BERT for zero-shot english idiomaticity detection 

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.21.2-py3-none-any.whl (4.7 MB)


In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/24/2022 23:38:57 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/24/2022 23:38:57 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Baseline PT : Monolingual BERT for zero-shot portuguese idiomaticity detection 

In [ ]:
# Portuguese language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/25/2022 00:51:23 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 00:51:23 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Baseline Multi : Multilingual BERT for zero-shot multilingual idiomaticity detection

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/ZeroShot/0/ \
    	--seed 0 \
    	--train_file      Data/ZeroShot/train.csv \
    	--validation_file Data/ZeroShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/25/2022 02:07:53 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 02:07:53 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Task+Lang adapter-based BERT for zero-shot english idiomaticity detection

In [21]:
# !pip install adapter-transformers
!pip install git+https://github.com/adapter-hub/adapter-transformers.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/adapter-hub/adapter-transformers.git to /tmp/pip-req-build-2ssu_vmn
  Running command git clone -q https://github.com/adapter-hub/adapter-transformers.git /tmp/pip-req-build-2ssu_vmn
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 5.3 MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-3.1.0a0-py3-none-any.whl size=4268660 sha256=b7881ca8d65c079a29e8f134949ca54d8a72d85f6871da410f03dfe9b7039c20
  Stored in directory: /tmp/pip-ephem-wheel-cache-sy78b6gu/wheels/0e/10/59/2e0642953eade6187066db14e31def5345c4ed7d757d32de99
Successfully built adapter-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.9.0
    Uninstalling huggingface-hub-0.9.0:
      Successfully unins

In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/0/ \
  --overwrite_output_dir \
  --seed 0 \
  --train_file      Data/ZeroShot/EN/train.csv \
  --validation_file Data/ZeroShot/EN/dev.csv \
	--evaluation_strategy "epoch" \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 03:57:02 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 03:57:02 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for zero-shot portuguese idiomaticity detection

In [22]:
# Portuguese language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/0/ \
  --overwrite_output_dir \
  --seed 0 \
  --train_file      Data/ZeroShot/PT/train.csv \
  --validation_file Data/ZeroShot/PT/dev.csv \
	--evaluation_strategy "epoch" \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 09:24:31 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 09:24:31 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for zero-shot EN-PT idiomatic knowledge transfer

In [23]:
# Transfer from English to Portuguese
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/1/ \
  --seed 1 \
  --train_file      Data/ZeroShot/EN/train.csv \
  --validation_file Data/ZeroShot/PT/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 05:08:42 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 05:08:42 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for zero-shot PT-EN idiomatic knowledge transfer

In [30]:
# Transfer from Portuguese to English
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/ZeroShot/1/ \
  --seed 1 \
  --train_file      Data/ZeroShot/PT/train.csv \
  --validation_file Data/ZeroShot/EN/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 10:25:16 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 10:25:16 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

# One Shot Setting

## Baseline EN : Monolingual BERT for one-shot english idiomaticity detection

In [ ]:
!pip install transformers

In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/25/2022 00:13:19 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 00:13:19 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Baseline PT : Monolingual BERT for one-shot portuguese idiomaticity detection

In [ ]:
# Portuguese language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/25/2022 01:05:26 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 01:05:26 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Baseline Multi : Multilingual BERT for one-shot multilingual idiomaticity detection 

In [ ]:
# Multi language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro.py \
    	--model_name_or_path 'bert-base-multilingual-cased' \
    	--do_train \
    	--do_eval \
    	--max_seq_length 128 \
    	--per_device_train_batch_size 32 \
    	--learning_rate 2e-5 \
    	--num_train_epochs 25 \
    	--evaluation_strategy "epoch" \
    	--output_dir models/OneShot/1/ \
    	--seed 1 \
    	--train_file      Data/OneShot/train.csv \
    	--validation_file Data/OneShot/dev.csv \
	    --evaluation_strategy "epoch" \
	    --save_strategy "epoch"  \
	    --load_best_model_at_end \
	    --metric_for_best_model "f1" \
	    --save_total_limit 1

08/25/2022 02:56:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 02:56:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precisi

## Task+Lang adapter-based BERT for one-shot english idiomaticity detection

In [ ]:
# !pip install adapter-transformers
!pip install git+https://github.com/adapter-hub/adapter-transformers.git

# https://adapterhub.ml/explore/text_lang/

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/adapter-hub/adapter-transformers.git to /tmp/pip-req-build-91j13268
  Running command git clone -q https://github.com/adapter-hub/adapter-transformers.git /tmp/pip-req-build-91j13268
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 86 kB 3.9 MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-3.1.0a0-py3-none-any.whl size=4268660 sha256=10c8024a801067a376d61baaf1e404eaafdef4a506d203aa19d0229c33ceed1c
  Stored in directory: /tmp/pip-ephem-wheel-cache-c1brb_3v/wheels/0e/10/59/2e0642953eade6187066db14e31def5345c4ed7d757d32de99
Successfully built adapter-transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.9.0
    Uninstalling huggingface-hub-0.9.0:
      Successfully unins

In [ ]:
# English language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/EN/train.csv \
  --validation_file Data/OneShot/EN/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 04:27:09 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 04:27:09 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for one-shot portuguese idiomaticity detection

In [23]:
# Portuguese language
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/PT/train.csv \
  --validation_file Data/OneShot/PT/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 09:37:53 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 09:37:53 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for one-shot EN-PT idiomatic knowledge transfer

In [32]:
# Transfer from English to Portuguese
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/EN/train.csv \
  --validation_file Data/OneShot/PT/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 10:57:15 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 10:57:15 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i

## Task+Lang adapter-based BERT for one-shot PT-EN idiomatic knowledge transfer

In [31]:
# Transfer from Portueguese to English
!python /content/AStitchInLanguageModels/Dataset/Task2/Utils/run_glue_f1_macro_adapters.py \
  --model_name_or_path 'bert-base-multilingual-cased' \
  --do_train \
  --do_eval \
  --max_seq_length 128 \
  --per_device_train_batch_size 32 \
  --learning_rate 1e-4 \
  --num_train_epochs 25 \
  --evaluation_strategy "epoch" \
  --output_dir models/OneShot/1/ \
  --seed 1 \
  --train_file      Data/OneShot/PT/train.csv \
  --validation_file Data/OneShot/EN/dev.csv \
  --overwrite_output_dir \
	--save_strategy "epoch"  \
	--load_best_model_at_end \
	--metric_for_best_model "f1" \
	--save_total_limit 1 \
  --train_adapter \
  --adapter_config pfeiffer

08/25/2022 10:39:01 - WARNING - __main__ - Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
08/25/2022 10:39:01 - INFO - __main__ - Training/evaluation parameters TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=IntervalStrategy.EPOCH,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_i